In [0]:
catalog = dbutils.widgets.get('catalog')
schema = dbutils.widgets.get('schema')

In [0]:
%sql
MERGE INTO ${catalog}.${schema}.pricehistory_silver AS target
USING (
  SELECT *
  FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY zpid, date ORDER BY file_name DESC) as rn
    FROM ${catalog}.${schema}.price_bronze
  ) AS deduped
  WHERE rn = 1
) AS source
ON target.zpid = source.zpid AND target.date = to_timestamp(source.date)

WHEN MATCHED THEN
  UPDATE SET
    target.event = source.event,
    target.price = source.price,
    target.priceChangeRate = source.priceChangeRate,
    target.pricePerSquareFoot = source.pricePerSquareFoot,
    target.file_name = source.file_name,
    target.Md5_hash = md5(concat_ws('', source.zpid, to_timestamp(source.date), source.event, source.price, source.priceChangeRate, source.pricePerSquareFoot, source.file_name))

WHEN NOT MATCHED THEN
  INSERT (zpid, date, event, price, priceChangeRate, pricePerSquareFoot, file_name, Md5_hash)
  VALUES (
    source.zpid,
    to_timestamp(source.date),
    source.event,
    source.price,
    source.priceChangeRate,
    source.pricePerSquareFoot,
    source.file_name,
    md5(concat_ws('', source.zpid, to_timestamp(source.date), source.event, source.price, source.priceChangeRate, source.pricePerSquareFoot, source.file_name))
  )


In [0]:
%sql
select * from ${catalog}.${schema}.pricehistory_silver